# Train YOLOv11n for Golf Ball Detection

Fine-tunes YOLOv11 nano on golf ball datasets from Roboflow Universe.

**Available Datasets:**
- `anna-gaming` - 17,460 images (recommended - best balance)
- `golf-ball-detection` - 5,785 images (faster training)
- `ratsstech` - 1,383 images (quick test)
- `golf-ball-tracker` - 458 images (smallest)

**Instructions:**
1. Open in Google Colab
2. Runtime > Change runtime type > GPU (T4 or better)
3. Get a free Roboflow API key at https://app.roboflow.com/settings/api
4. Run all cells
5. Download the trained model at the end

Training time: ~30-60 min depending on dataset size and GPU

In [ ]:
# Install dependencies
!pip install roboflow ultralytics -q
print("Dependencies installed!")

In [ ]:
# Configuration - EDIT THESE VALUES
ROBOFLOW_API_KEY = ""  # Get from https://app.roboflow.com/settings/api

# Dataset mode: "single" or "combined"
MODE = "combined"  # Recommended: combines multiple smaller datasets

# For single mode, choose one:
DATASET = "golf-ball-detection"  # 5,785 images

# For combined mode, these datasets will be merged:
COMBINE_DATASETS = [
    "golf-ball-detection",  # 5,785 images
    "ratsstech",            # 1,383 images  
    "golf-ball-tracker",    # 458 images
]
# Total: ~7,600 images - trains in ~20-30 min

# Training settings
EPOCHS = 100
IMAGE_SIZE = 640
BATCH_SIZE = 16  # Reduce to 8 if you get OOM errors

In [ ]:
# Download dataset(s) from Roboflow Universe
import os
import shutil
import yaml
from roboflow import Roboflow

rf = Roboflow(api_key=ROBOFLOW_API_KEY)

# Dataset configurations - workspace/project from Universe URLs
DATASETS = {
    "anna-gaming": {
        "workspace": "anna-gaming",
        "project": "golfball",
    },
    "golf-ball-detection": {
        "workspace": "golf-ball-detection",
        "project": "golf-ball-detection-hii2e",
    },
    "golf-ball-tracker": {
        "workspace": "golf-balls",
        "project": "golf-ball-tracker-sksye",
    },
    "ratsstech": {
        "workspace": "ratsstech-tjn5n",
        "project": "golf-ball-detector-9cehw",
    }
}

def download_dataset(ds_name):
    """Download a dataset, auto-detecting the latest version."""
    config = DATASETS[ds_name]
    project = rf.workspace(config["workspace"]).project(config["project"])
    
    # Get all versions and find the latest
    versions = project.versions()
    if not versions:
        raise RuntimeError(f"No versions found for {ds_name}")
    
    # Get the last version object from the list (latest)
    latest = versions[-1]
    print(f"  Found {len(versions)} version(s), using latest")
    
    return latest.download("yolov11")

if MODE == "single":
    # Download single dataset
    print(f"Downloading {DATASET}...")
    dataset = download_dataset(DATASET)
    dataset_location = dataset.location
    print(f"\nDataset: {DATASET}")
    print(f"Location: {dataset_location}")

else:
    # Download and combine multiple datasets
    print("Downloading and combining datasets...")
    combined_dir = os.path.abspath("combined_dataset")  # Use absolute path
    os.makedirs(f"{combined_dir}/train/images", exist_ok=True)
    os.makedirs(f"{combined_dir}/train/labels", exist_ok=True)
    os.makedirs(f"{combined_dir}/valid/images", exist_ok=True)
    os.makedirs(f"{combined_dir}/valid/labels", exist_ok=True)
    
    total_train = 0
    total_valid = 0
    
    for ds_name in COMBINE_DATASETS:
        print(f"\nDownloading {ds_name}...")
        dataset = download_dataset(ds_name)
        
        # Copy train images and labels
        src_train_img = f"{dataset.location}/train/images"
        src_train_lbl = f"{dataset.location}/train/labels"
        if os.path.exists(src_train_img):
            for f in os.listdir(src_train_img):
                shutil.copy(f"{src_train_img}/{f}", f"{combined_dir}/train/images/{ds_name}_{f}")
                total_train += 1
        if os.path.exists(src_train_lbl):
            for f in os.listdir(src_train_lbl):
                shutil.copy(f"{src_train_lbl}/{f}", f"{combined_dir}/train/labels/{ds_name}_{f}")
        
        # Copy valid images and labels
        src_valid_img = f"{dataset.location}/valid/images"
        src_valid_lbl = f"{dataset.location}/valid/labels"
        if os.path.exists(src_valid_img):
            for f in os.listdir(src_valid_img):
                shutil.copy(f"{src_valid_img}/{f}", f"{combined_dir}/valid/images/{ds_name}_{f}")
                total_valid += 1
        if os.path.exists(src_valid_lbl):
            for f in os.listdir(src_valid_lbl):
                shutil.copy(f"{src_valid_lbl}/{f}", f"{combined_dir}/valid/labels/{ds_name}_{f}")
    
    # Create combined data.yaml with ABSOLUTE paths
    data_yaml = {
        "train": f"{combined_dir}/train/images",
        "val": f"{combined_dir}/valid/images",
        "nc": 1,
        "names": ["golf-ball"]
    }
    with open(f"{combined_dir}/data.yaml", "w") as f:
        yaml.dump(data_yaml, f)
    
    dataset_location = combined_dir
    print(f"\n{'='*40}")
    print(f"Combined dataset created!")
    print(f"Training images: {total_train}")
    print(f"Validation images: {total_valid}")
    print(f"Location: {dataset_location}")
    print(f"{'='*40}")

In [ ]:
# Check dataset structure
import os

print("Dataset contents:")
!ls -la "{dataset_location}"
print("\ndata.yaml:")
!cat "{dataset_location}/data.yaml"

# Count images
train_path = f"{dataset_location}/train/images"
valid_path = f"{dataset_location}/valid/images"
train_count = len(os.listdir(train_path)) if os.path.exists(train_path) else 0
valid_count = len(os.listdir(valid_path)) if os.path.exists(valid_path) else 0
print(f"\nTraining images: {train_count}")
print(f"Validation images: {valid_count}")
print(f"Total: {train_count + valid_count}")

In [ ]:
from ultralytics import YOLO

# Load YOLOv11 nano (optimized for edge devices)
model = YOLO('yolo11n.pt')
print("Loaded YOLOv11n base model")
print(f"Parameters: {sum(p.numel() for p in model.model.parameters()):,}")

In [ ]:
# Train the model
results = model.train(
    data=f"{dataset_location}/data.yaml",
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    patience=20,          # Early stopping
    device=0,
    workers=2,
    project='golf_ball_detector',
    name='yolo11n_golfball',
    pretrained=True,
    optimizer='AdamW',
    lr0=0.001,
    augment=True,
    hsv_h=0.015,          # Hue augmentation
    hsv_s=0.7,            # Saturation augmentation  
    hsv_v=0.4,            # Value augmentation
    degrees=15,           # Rotation
    scale=0.5,            # Scale augmentation
    fliplr=0.5,           # Horizontal flip
    mosaic=1.0,           # Mosaic augmentation
)

In [ ]:
# Evaluate the model
metrics = model.val()

print("\n" + "="*50)
print("FINAL METRICS")
print("="*50)
print(f"mAP50:     {metrics.box.map50:.3f}")
print(f"mAP50-95:  {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall:    {metrics.box.mr:.3f}")
print("="*50)

if metrics.box.map50 > 0.8:
    print("\nExcellent! Model is ready for production.")
elif metrics.box.map50 > 0.6:
    print("\nGood performance. Consider more training data for improvement.")
else:
    print("\nModel may need more training or data augmentation.")

In [ ]:
# Export to multiple formats for Raspberry Pi
import shutil

best_model = YOLO('golf_ball_detector/yolo11n_golfball/weights/best.pt')

# PyTorch format (most compatible)
shutil.copy('golf_ball_detector/yolo11n_golfball/weights/best.pt', 'golf_ball_yolo11n.pt')
print("Saved PyTorch model: golf_ball_yolo11n.pt")

# ONNX format (good CPU performance)
best_model.export(format='onnx', imgsz=IMAGE_SIZE, simplify=True)
shutil.copy('golf_ball_detector/yolo11n_golfball/weights/best.onnx', 'golf_ball_yolo11n.onnx')
print("Saved ONNX model: golf_ball_yolo11n.onnx")

# NCNN format (optimized for ARM/Pi)
best_model.export(format='ncnn', imgsz=IMAGE_SIZE)
!zip -r -q golf_ball_yolo11n_ncnn.zip golf_ball_detector/yolo11n_golfball/weights/best_ncnn_model/
print("Saved NCNN model: golf_ball_yolo11n_ncnn.zip")

print("\nAll exports complete!")

In [ ]:
# Test on sample images
import glob
from IPython.display import Image, display

# Get a few validation images
val_images = glob.glob(f"{dataset_location}/valid/images/*")[:3]

print("Testing on validation images:")
for img_path in val_images:
    results = best_model.predict(img_path, conf=0.3, save=True, project='test_results')
    print(f"Detected {len(results[0].boxes)} golf ball(s) in {img_path.split('/')[-1]}")

# Show results
result_images = glob.glob('test_results/predict*/*.jpg')[:3]
for img in result_images:
    display(Image(filename=img, width=400))

In [ ]:
# Download trained models
from google.colab import files

print("Downloading models...")
print("\n1. PyTorch model (.pt) - use with ultralytics")
files.download('golf_ball_yolo11n.pt')

print("\n2. ONNX model (.onnx) - use with onnxruntime")
files.download('golf_ball_yolo11n.onnx')

print("\n3. NCNN model (.zip) - optimized for ARM/Raspberry Pi")
files.download('golf_ball_yolo11n_ncnn.zip')

print("\nAll downloads complete!")

## Deploy to Raspberry Pi

```bash
# Copy model to Pi
scp golf_ball_yolo11n.onnx pi@<ip>:~/openlaunch/models/

# Or for NCNN (faster on Pi)
scp golf_ball_yolo11n_ncnn.zip pi@<ip>:~/openlaunch/models/
ssh pi@<ip> 'cd ~/openlaunch/models && unzip golf_ball_yolo11n_ncnn.zip'

# Start OpenLaunch with the new model
cd ~/openlaunch
./scripts/start-kiosk.sh --camera-model models/golf_ball_yolo11n.onnx

# Or with NCNN
./scripts/start-kiosk.sh --camera-model models/golf_ball_detector/yolo11n_golfball/weights/best_ncnn_model
```

### Recommended Model Format
- **ONNX** - Best compatibility, good performance
- **NCNN** - Fastest on Raspberry Pi ARM CPU
- **PyTorch (.pt)** - Most flexible, requires more memory